# Storing API Requests in MongoDB
What we want to do is to store the requests in such a way that we can make use of the objects and classes to analyse the league and results. Then when we are using these objects, we aren't calling the API again, but rather querying our database that will be filled using a script that makes sure that we don't exceed the API request limit defined by the account type.    

This builds on the other notebooks that have written that show the basics of API calls and MongoDB. 

In [127]:
# Import packages 
import json
import os
import pandas as pd
import pymongo
import requests
from pprint import pprint 

In [128]:
# Get the database details from the conda environment 
mongodb_password = os.environ.get("mongodb_password")
mongodb_user = os.environ.get("mongodb_user")

When we end up automating this, we are going to want to set a hard limit and control the requests, throttling our access to make sure we don't incure any additional costs for the tier. 

In [129]:
# Define max requests for this run 
api_max_requests = 10

In [130]:
# Access the database client and define database that we want to store the results in
client = pymongo.MongoClient("mongodb+srv://" + mongodb_user + ":" + mongodb_password + "@basiccluster-6s0er.mongodb.net/test?retryWrites=true&w=majority")
db = client.football

In [131]:
# Get the API details from the conda environment 
headers = {
    'x-rapidapi-host': os.environ.get("api_host"), 
    'x-rapidapi-key': os.environ.get("api_key") 
    }

I find that defining a simple `api_call` helper function here to be very useful so that we aren't continually writing the same code, and it makes the code easier to follow and read what the process is actually doing. 

In [132]:
def api_call(url):
    response = requests.request("GET", url, headers=headers)
    return json.loads(response.text)

For now, I'm going to focus on the Premier League. If required, we can extend and restructure.

In [12]:
# Define the base URL for the API requests
base_url = 'https://api-football-v1.p.rapidapi.com/v2/'
premier_league_id = '524'
fixtures_url = base_url + 'fixtures/league/' + premier_league_id

# Call the league fixture list and compare to current version
fixtures = api_call(fixtures_url)

In [28]:
print(fixtures.keys()) # Master level 
print(fixtures['api'].keys()) # Split into results and fixtures
print(fixtures['api']['results']) # Note this is the just the size of the response
print(fixtures['api']['fixtures'][0]) # Example fixture
print("\nTotal Fixtures: %s" % len(fixtures['api']['fixtures'])) # Total fixtures 

dict_keys(['api'])
dict_keys(['results', 'fixtures'])
380
{'fixture_id': 157015, 'league_id': 524, 'league': {'name': 'Premier League', 'country': 'England', 'logo': 'https://media.api-football.com/leagues/2.png', 'flag': 'https://media.api-football.com/flags/gb.svg'}, 'event_date': '2019-08-09T19:00:00+00:00', 'event_timestamp': 1565377200, 'firstHalfStart': 1565377200, 'secondHalfStart': 1565380800, 'round': 'Regular Season - 1', 'status': 'Match Finished', 'statusShort': 'FT', 'elapsed': 90, 'venue': 'Anfield (Liverpool)', 'referee': 'Michael Oliver, England', 'homeTeam': {'team_id': 40, 'team_name': 'Liverpool', 'logo': 'https://media.api-football.com/teams/40.png'}, 'awayTeam': {'team_id': 71, 'team_name': 'Norwich', 'logo': 'https://media.api-football.com/teams/71.png'}, 'goalsHomeTeam': 4, 'goalsAwayTeam': 1, 'score': {'halftime': '4-0', 'fulltime': '4-1', 'extratime': None, 'penalty': None}}

Total Fixtures: 380


We are therefore only interested in the fixtures list that we can using the syntax below. 

In [29]:
fixtures = fixtures['api']['fixtures']

We are then going to insert all of these fixtures into the `fixtures` collection of our MongoDB database. 

In [42]:
db.fixtures.insert_many(fixtures); 

In [50]:
# Take a quick look at one of the fixtures
pprint(db.fixtures.find_one({}))

{'_id': ObjectId('5de6cd1367605fece82bbb21'),
 'awayTeam': {'logo': 'https://media.api-football.com/teams/71.png',
              'team_id': 71,
              'team_name': 'Norwich'},
 'elapsed': 90,
 'event_date': '2019-08-09T19:00:00+00:00',
 'event_timestamp': 1565377200,
 'firstHalfStart': 1565377200,
 'fixture_id': 157015,
 'goalsAwayTeam': 1,
 'goalsHomeTeam': 4,
 'homeTeam': {'logo': 'https://media.api-football.com/teams/40.png',
              'team_id': 40,
              'team_name': 'Liverpool'},
 'league': {'country': 'England',
            'flag': 'https://media.api-football.com/flags/gb.svg',
            'logo': 'https://media.api-football.com/leagues/2.png',
            'name': 'Premier League'},
 'league_id': 524,
 'referee': 'Michael Oliver, England',
 'round': 'Regular Season - 1',
 'score': {'extratime': None,
           'fulltime': '4-1',
           'halftime': '4-0',
           'penalty': None},
 'secondHalfStart': 1565380800,
 'status': 'Match Finished',
 'statusShor

## Filtering the fixtures

We have inserted the 380 fixtures for the premier league for this season. Going forward we are going to want to call this again and then just update those that have changed i.e. games that have been played. 

In [51]:
completed_matches = list(db.fixtures.find({'status': 'Match Finished'}))

In [52]:
len(completed_matches)

140

There have been 140 completed_matches so far this season according to our records.  

These are the matches that we are going to want to further get the stats for and store against the fixture. 

In [53]:
postponed_matches = list(db.fixtures.find(
    {"$and": [
        {"status": {
            "$ne": 'Match Finished'
        }},
        {"status": {
            "$ne": 'Not Started'
        }}
    ]}
))

In [55]:
pprint(postponed_matches)

[{'_id': ObjectId('5de6cd1367605fece82bbbad'),
  'awayTeam': {'logo': 'https://media.api-football.com/teams/35.png',
               'team_id': 35,
               'team_name': 'Bournemouth'},
  'elapsed': 60,
  'event_date': '2019-12-03T19:30:00+00:00',
  'event_timestamp': 1575401400,
  'firstHalfStart': 1575401400,
  'fixture_id': 157163,
  'goalsAwayTeam': 0,
  'goalsHomeTeam': 0,
  'homeTeam': {'logo': 'https://media.api-football.com/teams/52.png',
               'team_id': 52,
               'team_name': 'Crystal Palace'},
  'league': {'country': 'England',
             'flag': 'https://media.api-football.com/flags/gb.svg',
             'logo': 'https://media.api-football.com/leagues/2.png',
             'name': 'Premier League'},
  'league_id': 524,
  'referee': 'Anthony Taylor, England',
  'round': 'Regular Season - 15',
  'score': {'extratime': None,
            'fulltime': None,
            'halftime': '0-0',
            'penalty': None},
  'secondHalfStart': 1575405000,
  'sta

You can see that when I called this API, there were 2 games in progress, and 1 game that came up from this search that has been postponed. 

In [57]:
incomplete_matches = list(db.fixtures.find({'status': {
            "$ne": 'Match Finished'
        }}, {'fixture_id'}))

Get the fixture IDs for those matches that haven't been completed yet. 

In [58]:
incomplete_fixture_ids = [match['fixture_id'] for match in incomplete_matches]

# Using indexes to maintain data integrity
Going forward we are going to want to use what we have stored, and work out which fixtures need updating. Again, this is to do with limiting the requests that we are going to be doing to the account.  

One particular thing that I want to avoid is duplicating the fixtures themselves. We want to be able to update the information for each fixture, but never to duplicate them. A unique index should prevent us from doing this by accident as it will fail the insert statement. 

In [59]:
db.fixtures.create_index('fixture_id', unique=True, name='fixture_id_pk', default_language='english')

'fixture_id_pk'

In [68]:
print("First fixture id: %d\n" % db.fixtures.find_one()['fixture_id'])
test_fixture = db.fixtures.find_one({'fixture_id': 157015})
pprint(test_fixture)

First fixture id: 157015

{'_id': ObjectId('5de6cd1367605fece82bbb21'),
 'awayTeam': {'logo': 'https://media.api-football.com/teams/71.png',
              'team_id': 71,
              'team_name': 'Norwich'},
 'elapsed': 90,
 'event_date': '2019-08-09T19:00:00+00:00',
 'event_timestamp': 1565377200,
 'firstHalfStart': 1565377200,
 'fixture_id': 157015,
 'goalsAwayTeam': 1,
 'goalsHomeTeam': 4,
 'homeTeam': {'logo': 'https://media.api-football.com/teams/40.png',
              'team_id': 40,
              'team_name': 'Liverpool'},
 'league': {'country': 'England',
            'flag': 'https://media.api-football.com/flags/gb.svg',
            'logo': 'https://media.api-football.com/leagues/2.png',
            'name': 'Premier League'},
 'league_id': 524,
 'referee': 'Michael Oliver, England',
 'round': 'Regular Season - 1',
 'score': {'extratime': None,
           'fulltime': '4-1',
           'halftime': '4-0',
           'penalty': None},
 'secondHalfStart': 1565380800,
 'status': 'Mat

In [69]:
db.fixtures.insert_one(test_fixture)

DuplicateKeyError: E11000 duplicate key error collection: football.fixtures index: _id_ dup key: { : ObjectId('5de6cd1367605fece82bbb21') }

In [74]:
print("Total fixtures in db: %d" % len(list(db.fixtures.find({}))))

Total fixtures in db: 380


We can see that we haven't inserted that duplicate fixture as the index failed the insert due to a `DuplicateKeyError`. 

# Identifying records to update 
We are going to want to update the fixtures that we have with the post-match [statistics](https://www.api-football.com/documentation#fixtures-statistics) when they become available.  

At the moment the `fixtures` actually represent final results, and upcoming fixtures. We can identify which fixtures we don't have statistics for by using the `'$exists'` syntax. 

In [80]:
fixture_without_stats = db.fixtures.find({'statistics': { '$exists' : False }})[0]
pprint(fixture_without_stats)
print(fixture_without_stats.keys())

{'_id': ObjectId('5de6cd1367605fece82bbb21'),
 'awayTeam': {'logo': 'https://media.api-football.com/teams/71.png',
              'team_id': 71,
              'team_name': 'Norwich'},
 'elapsed': 90,
 'event_date': '2019-08-09T19:00:00+00:00',
 'event_timestamp': 1565377200,
 'firstHalfStart': 1565377200,
 'fixture_id': 157015,
 'goalsAwayTeam': 1,
 'goalsHomeTeam': 4,
 'homeTeam': {'logo': 'https://media.api-football.com/teams/40.png',
              'team_id': 40,
              'team_name': 'Liverpool'},
 'league': {'country': 'England',
            'flag': 'https://media.api-football.com/flags/gb.svg',
            'logo': 'https://media.api-football.com/leagues/2.png',
            'name': 'Premier League'},
 'league_id': 524,
 'referee': 'Michael Oliver, England',
 'round': 'Regular Season - 1',
 'score': {'extratime': None,
           'fulltime': '4-1',
           'halftime': '4-0',
           'penalty': None},
 'secondHalfStart': 1565380800,
 'status': 'Match Finished',
 'statusShor

In [116]:
fixtures_to_complete = db.fixtures.find({'statistics': { '$exists' : False }, 
                 'statusShort':'FT'})

In [117]:
pd.DataFrame(fixtures_to_complete).shape

(140, 19)

This is the list of completed fixtures that we don't have the statistics for. We are best off writing a python script that will handle filling the detail statistics in for us, with a hard limit of how many requests we are willing to do for that run. 

# Adding league level collection
There are a number of API calls that can be done at league level, so it makes sense that we should store these in a `db.leagues` collection. The fixtures can easily be obtained as the league information is stored within the fixture document. 

In [133]:
league = api_call("https://api-football-v1.p.rapidapi.com/v2/teams/league/%d" % 524)

In [142]:
pd.DataFrame([{'team_name': x['name'], 'venue_name': x['venue_name']} for x in league['api']['teams']])

,team_name,venue_name
0,Liverpool,Anfield
1,Norwich,Carrow Road
2,West Ham,London Stadium
3,Manchester City,Etihad Stadium
4,Bournemouth,Vitality Stadium
5,Sheffield Utd,Bramall Lane
6,Burnley,Turf Moor
7,Southampton,St. Mary's Stadium
8,Crystal Palace,Selhurst Park
9,Everton,Goodison Park


In [144]:
standings = api_call("https://api-football-v1.p.rapidapi.com/v2/leagueTable/%d" % 524)

In [159]:
print(standings['api']['standings'][0])

[{'rank': 1, 'team_id': 40, 'teamName': 'Liverpool', 'logo': 'https://media.api-football.com/teams/40.png', 'group': 'Premier League', 'forme': 'WWWWW', 'status': 'same', 'description': 'Promotion - Champions League (Group Stage)', 'all': {'matchsPlayed': 14, 'win': 13, 'draw': 1, 'lose': 0, 'goalsFor': 32, 'goalsAgainst': 12}, 'home': {'matchsPlayed': 7, 'win': 7, 'draw': 0, 'lose': 0, 'goalsFor': 19, 'goalsAgainst': 7}, 'away': {'matchsPlayed': 7, 'win': 6, 'draw': 1, 'lose': 0, 'goalsFor': 13, 'goalsAgainst': 5}, 'goalsDiff': 20, 'points': 40, 'lastUpdate': '2019-12-03'}, {'rank': 2, 'team_id': 46, 'teamName': 'Leicester', 'logo': 'https://media.api-football.com/teams/46.png', 'group': 'Premier League', 'forme': 'WWWWW', 'status': 'same', 'description': 'Promotion - Champions League (Group Stage)', 'all': {'matchsPlayed': 14, 'win': 10, 'draw': 2, 'lose': 2, 'goalsFor': 33, 'goalsAgainst': 9}, 'home': {'matchsPlayed': 7, 'win': 6, 'draw': 1, 'lose': 0, 'goalsFor': 16, 'goalsAgainst'

In [160]:
top_scorers = api_call("https://api-football-v1.p.rapidapi.com/v2/topscorers/%d" % 524)

In [169]:
ts = pd.DataFrame(top_scorers['api']['topscorers'])
pd.concat([ts, ts['games'].apply(pd.Series)], axis = 1).drop('games', axis = 1)

,player_id,player_name,firstname,lastname,position,nationality,team_id,team_name,goals,shots,penalty,cards,appearences,minutes_played
0,18788,J. Vardy,Jamie,Vardy,Attacker,England,46,Leicester,"{'total': 12, 'assists': 3, 'conceded': None}","{'total': 31, 'on': 20}","{'won': 1, 'commited': None, 'success': 2, 'mi...","{'yellow': 1, 'second_yellow': 0, 'red': 0}",13,1170
1,19194,T. Abraham,Kevin Oghenetega Tamaraebi,Bakumo-Abraham,Attacker,England,49,Chelsea,"{'total': 10, 'assists': 2, 'conceded': None}","{'total': 40, 'on': 20}","{'won': None, 'commited': None, 'success': 0, ...","{'yellow': 2, 'second_yellow': 0, 'red': 0}",13,970
2,642,S. Agüero,Sergio Leonel,Agüero del Castillo,Attacker,Argentina,50,Manchester City,"{'total': 9, 'assists': 2, 'conceded': None}","{'total': 42, 'on': 12}","{'won': None, 'commited': None, 'success': 2, ...","{'yellow': 0, 'second_yellow': 0, 'red': 0}",12,813
3,304,S. Mané,Sadio,Mané,Attacker,Senegal,40,Liverpool,"{'total': 8, 'assists': 2, 'conceded': None}","{'total': 33, 'on': 17}","{'won': 2, 'commited': None, 'success': 0, 'mi...","{'yellow': 1, 'second_yellow': 0, 'red': 0}",14,1149
4,645,R. Sterling,Raheem Shaquille,Sterling,Attacker,England,50,Manchester City,"{'total': 8, 'assists': 1, 'conceded': None}","{'total': 42, 'on': 15}","{'won': None, 'commited': None, 'success': 0, ...","{'yellow': 4, 'second_yellow': 0, 'red': 0}",13,1152
5,1465,P. Aubameyang,Pierre-Emerick,Aubameyang,Attacker,Gabon,42,Arsenal,"{'total': 8, 'assists': None, 'conceded': None}","{'total': 34, 'on': 14}","{'won': None, 'commited': None, 'success': 0, ...","{'yellow': 2, 'second_yellow': 0, 'red': 0}",13,1170
6,18955,D. Ings,Daniel William John,Ings,Attacker,England,41,Southampton,"{'total': 7, 'assists': 1, 'conceded': None}","{'total': 30, 'on': 13}","{'won': 1, 'commited': None, 'success': 0, 'mi...","{'yellow': 1, 'second_yellow': 0, 'red': 0}",14,919
7,184,H. Kane,Harry,Kane,Attacker,England,47,Tottenham,"{'total': 7, 'assists': 1, 'conceded': None}","{'total': 36, 'on': 13}","{'won': None, 'commited': None, 'success': 1, ...","{'yellow': 1, 'second_yellow': 0, 'red': 0}",13,1165
8,909,M. Rashford,Marcus,Rashford,Attacker,England,33,Manchester United,"{'total': 7, 'assists': 4, 'conceded': None}","{'total': 40, 'on': 16}","{'won': 2, 'commited': None, 'success': 2, 'mi...","{'yellow': 2, 'second_yellow': 0, 'red': 0}",13,1130
9,19085,T. Pukki,Teemu,Pukki,Attacker,Finland,71,Norwich,"{'total': 6, 'assists': 3, 'conceded': None}","{'total': 35, 'on': 18}","{'won': None, 'commited': None, 'success': 0, ...","{'yellow': 2, 'second_yellow': 0, 'red': 0}",13,1157
